# Knowledge Graph Construction

In [ ]:
! pip install pandas langchain_neo4j langchain_core langchain_ollama

**Connection to Neo4j**

In [20]:
# 1. Importing Required Libraries
from neo4j import GraphDatabase
from langchain_neo4j import Neo4jGraph

# 2. Connect to Neo4j
NEO4J_URI="neo4j+s://07561b92.databases.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="2wQMIQq_1cANjkh837eMSHUsYBH2F-XLialhvPmM3Bw"

URI = NEO4J_URI
AUTH = (NEO4J_USERNAME,NEO4J_PASSWORD) #AUTH = ("<Username>", "<Password>")
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()


graph=Neo4jGraph(url=NEO4J_URI,username=NEO4J_USERNAME,password=NEO4J_PASSWORD)
print(graph)
print("Connected to Neo4j graph.")

Connected to Neo4j graph.


**Clear the graph by removing all nodes & relations**

In [21]:
graph.query("""MATCH (n) DETACH DELETE n   // Delete all existing nodes and relationships""")
print("Suceesfully deleted all nodes and relations.")

Suceesfully deleted all nodes and relations.


**Load csv data**

In [22]:
import pandas as pd

df=pd.read_csv("C:/Users/admin/OneDrive/Desktop/Symptoms_based_disease_prediction/Datasets/preprocessed_data.csv")
df.head()

,symptoms,prognosis
0,"itching, skin_rash, nodal_skin_eruptions, disc...",Fungal Infection
1,"skin_rash, nodal_skin_eruptions, dischromic _p...",Fungal Infection
2,"itching, nodal_skin_eruptions, dischromic _pat...",Fungal Infection
3,"itching, skin_rash, dischromic _patches",Fungal Infection
4,"itching, skin_rash, nodal_skin_eruptions",Fungal Infection


**Push data into KG**

In [24]:
# 3. Seed the Database (Load data from CSV)
for index, row in df.iterrows():
    prognosis = row['prognosis']
    symptoms = row['symptoms'].split(', ')

    # Create or update disease node (prognosis)
    graph.query(f"""MERGE (d:Disease {{name: "{prognosis}"}})""")

    # Iterate over symptoms to create nodes and relationships
    for symptom in symptoms:       
        # Create or update symptom node                            
        graph.query(f"""MERGE (s:Symptom {{name: "{symptom}"}}) """)         
       
        # Create or update relationship between symptom and disease
        graph.query(f"""                    
        MATCH (s:Symptom {{name: "{symptom}"}}), (d:Disease {{name: "{prognosis}"}})
        MERGE (s)-[:HAS_SYMPTOM]->(d)
        """)


**Print Schema**

In [25]:
# 4. Refresh Schema (After inserting data, refresh the schema for changes)
graph.refresh_schema()
print(graph.schema)

Node properties:
Disease {name: STRING}
Symptom {name: STRING}
Relationship properties:

The relationships:
(:Symptom)-[:HAS_SYMPTOM]->(:Disease)


**KG check**

In [40]:
graph.query(
"""
MATCH (d:Disease)<-[:HAS_SYMPTOM]-(s:Symptom)
WHERE s.name IN ["itching", "skin_rash", "nodal_skin_eruptions"]
RETURN d.name AS Disease, COUNT(s) AS MatchingSymptoms
ORDER BY MatchingSymptoms DESC
LIMIT 5

""")

[{'Disease': 'Fungal Infection', 'MatchingSymptoms': 3},
 {'Disease': 'Drug Reaction', 'MatchingSymptoms': 2},
 {'Disease': 'Chickenpox', 'MatchingSymptoms': 2},
 {'Disease': 'Jaundice', 'MatchingSymptoms': 1},
 {'Disease': 'Chronic Cholestasis', 'MatchingSymptoms': 1}]